In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.io as pio



In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_unstained4"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\unstained_"
maskpath=basepath+"\\ExportedMasks"

num_images=10000

image_dim=64 #Dim of the final images

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_smallerfile2"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\smaller_file2_"
maskpath=basepath+"\\ExportedMasks"

num_images=10000

image_dim=64 #Dim of the final images

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_sample1"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\sample1_"
maskpath=basepath+"\\ExportedMasks"



image_dim=64 #Dim of the final images

nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"

In [ ]:
#Find cell names
cell_names=np.load(outpath+"\\image_ID.npy")
cell_names=[int(cell_name) for cell_name in cell_names]
image_dict={}

for cell_name in cell_names:
    image_dict[cell_name]={}


#Find Channels
names=[]
for entry in os.listdir(outpath): #Read all files
    if os.path.isfile(os.path.join(outpath, entry)):
        if entry!='image_ID.npy':
            names.append(entry)


channels=[name[:-4] for name in names if name[-4:]=='.npy']

print("Channels found:",channels)

data_dict={}
for channel in channels:
    data_dict[channel]=np.load(outpath+"\\"+channel+'.npy')

#Break up array

for channel in data_dict:
    dims=data_dict[channel].shape
    n=dims[0]//image_dim
    l=dims[1]//image_dim
    index=0
    for i in range(n):
        for j in range(l):
            img=data_dict[channel][i*image_dim:i*image_dim+image_dim,j*image_dim:j*image_dim+image_dim]
            image_dict[cell_names[index]][channel]=img
            index+=1



In [ ]:
channels=['Ch1',
 'Ch11',
 'Ch2',
 'Ch3',
 'Ch7']

In [ ]:
df=pd.read_csv(outpath+"\\cell_info.csv")

In [ ]:
df

In [ ]:
ch11=[]
ch2=[]
ch3=[]
ch7=[]
for cell in df['Cell_ID'].to_numpy():
    

    



In [ ]:
df[df['Cell_ID']==i]

In [ ]:
i=7503
d_img=image_dict[i]['Ch1']
for channel in channels[1:]:
    d_img=np.hstack((d_img,image_dict[i][channel]))
plt.imshow(d_img)
df[df['Cell_ID']==i]

In [ ]:
for i in range(100):
    d_img=image_dict[cell_names[i]]['Ch1']
    for channel in channels[1:]:
        d_img=np.hstack((d_img,image_dict[cell_names[i]][channel]))
    if i ==0:
        display_img=d_img
    else:
        display_img=np.vstack((display_img,d_img))

plt.figure(figsize=(10,10))
plt.imshow(display_img)
plt.axis('off')
    

In [ ]:
#Read in images
names=[]
for entry in os.listdir(readpath): #Read all photos
    if os.path.isfile(os.path.join(readpath, entry)):
        names.append(entry)
       # if len(names)>=num_images*3:
        #    break

channel_names=[channel_name[:-8] for channel_name in np.unique([name.split("_")[file_prefix.count("_")+1] for name in names])] 

print("Found",len(channel_names),"channels")
print(channel_names)

data_dict={}
for name in names:
    cell_num=name.split("_")[file_prefix.count("_")]
    if cell_num not in data_dict.keys():
        data_dict[cell_num]=[cv.imread(readpath+file_prefix+cell_num+'_'+channel_name+'.ome.tif',0  | cv.IMREAD_ANYDEPTH) for channel_name in channel_names]
        data_dict[cell_num]+=[cv.imread(maskpath+file_prefix+cell_num+'_'+channel_name+'.dmask.pgm',0 ) for channel_name in channel_names]

#13s

In [ ]:
plt.imshow(data_dict['0'][7])

In [ ]:
np.max(data_dict['0'][7])

In [ ]:
#Method 1 is based on the maximal fluorescent pixel

#Extract fluorescent features

features=np.zeros((len(cell_names),4))
for i,cells in enumerate(data_dict):
    features[i][0]=cells
    features[i][1]=0+np.max(data_dict[cells][2])#[data_dict[cells][7+2]==255]) #FITC very few pos
    features[i][2]=0+np.max(data_dict[cells][3])#[data_dict[cells][7+2]==255]) #PE  very few pos
    features[i][3]=0+np.max(data_dict[cells][1])#[data_dict[cells][7+2]==255]) #APC  60% pos

features_t=features>=120
colors=[]
for i in features_t:
    if sum(i[1:])==1:
        if i[1]==1:
            colors.append("FITC (Smooth Muscle Actin)")
        if i[2]==1:
            colors.append("PE (Stromal cells)")
        if i[3]==1:
            colors.append("APC (CD45)")
    else:
        colors.append("Other cell type")

features_relative=features[:,1:]
import plotly.express as px
fig = px.scatter_3d(x=features_relative[:,0],y=features_relative[:,1],z=features_relative[:,2],width=1000, height=1000,color=colors)

fig.update_layout(scene = dict(
                    xaxis_title='FITC',
                    yaxis_title='PE',
                    zaxis_title='APC'))

fig.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=0.3*2, y=0.2*2, z=0.8*2))

fig.update_traces(marker_size =2)
#fig.update_layout(scene_aspectmode='cube')

pio.renderers.default = "notebook"
fig.show()
#fig.write_html(outpath+"\\gating.html")

In [ ]:
my_dict={}
my_dict["FITC (Smooth Muscle Actin)"]=1
my_dict['APC (CD45)']=2
my_dict['PE (Stromal cells)']=3
my_dict['Other cell type']=0

In [ ]:
a=[1,2,3,9,0]

In [ ]:
b=np.max(np.delete(a,3))

In [ ]:
i

In [ ]:
#method 2 is based on the N maximal pixel intensities

#Extract fluorescent features
threshold=10  #Select the n brightest pixels of the image


features=np.zeros((len(cell_names),4))
for i,cells in enumerate(data_dict):
    features[i][0]=cells
    features[i][1]=np.mean(np.sort(data_dict[cells][2],axis=None)[-threshold:]) #FITC very few pos
    features[i][2]=np.mean(np.sort(data_dict[cells][3],axis=None)[-threshold:]) #PE  very few pos
    features[i][3]=np.mean(np.sort(data_dict[cells][1],axis=None)[-threshold:]) #APC  60% pos

features_t=features#>=120
colors=[]
for i in features_t:
    i=i[1:]

    n=np.argmax(i)
    colors_str=["Other cell type","FITC (Smooth Muscle Actin)","PE (Stromal cells)","APC (CD45)"]

    if i[n]>=150:
        if i[n]/np.sum(i)>0.45:
            colors.append(colors_str[n+1])
        else:
            colors.append(colors_str[0])
    else:
        colors.append(colors_str[0])


features_relative=features[:,1:]
import plotly.express as px
fig = px.scatter_3d(x=features_relative[:,0],y=features_relative[:,1],z=features_relative[:,2],width=1000, height=1000,color=colors)

fig.update_layout(scene = dict(
                    xaxis_title='FITC',
                    yaxis_title='PE',
                    zaxis_title='APC'))

#fig.update_layout(scene_aspectmode='manual',
 #                 scene_aspectratio=dict(x=0.3*2, y=0.2*2, z=0.8*2))

fig.update_traces(marker_size =2)
#fig.update_layout(scene_aspectmode='cube')

pio.renderers.default = "notebook"
fig.show()
fig.write_html(outpath+"\\gating.html")

In [ ]:
BUGS=[]
for image in data_dict:
    for i in data_dict[image]:
      #  print(i)
        try: 
            if i==None:
                BUGS.append(image)
        except:
            pass

In [ ]:
[data_dict.pop(BUG) for BUG in BUGS]

In [ ]:
df=pd.read_csv(outpath+"\\cell_info.csv")
df['Cell_ID2']=list(data_dict.keys())[:len(df)]
df['Cell_Type']=[my_dict[color] for color in colors][:len(df)]
df['Cell_Type_str']=colors[:len(df)]
df.to_csv(outpath+"\\cell_info.csv")

In [ ]:

plt.imshow(data_dict[cells][2])
plt.colorbar()
plt.show()
plt.imshow(data_dict[cells][3])
plt.colorbar()
plt.show()
plt.imshow(data_dict[cells][1])
plt.colorbar()
plt.show()




In [ ]:
features_t=features>=100

In [ ]:
colors=[]
for i in features_t:
    if sum(i[1:])==1:
        if i[1]==1:
            colors.append("red")
        if i[2]==1:
            colors.append("black")
        if i[3]==1:
            colors.append("green")
    else:
        colors.append("blue")

In [ ]:
features[:,1]-=np.min(features[:,1])
features[:,2]-=np.min(features[:,2])
features[:,3]-=np.min(features[:,3])


In [ ]:
print(features)

In [ ]:
plt.hist(features[:,1],bins=100);

In [ ]:
plt.hist(features[:,2],bins=100);

In [ ]:
plt.hist(features[:,3],bins=100);

In [ ]:
features[:,1]/=np.mean(features[:,1:],axis=1)
features[:,2]/=np.mean(features[:,1:],axis=1)
features[:,3]/=np.mean(features[:,1:],axis=1)

In [ ]:
np.mean(features[:,1:],axis=0).shape

In [ ]:
features_relative=features[:,1:]
#features_relative[:,0]/=np.sum(features[:,1:],axis=1)
#features_relative[:,1]/=np.sum(features[:,1:],axis=1)
#eatures_relative[:,2]/=np.sum(features[:,1:],axis=1)


In [ ]:
plt.scatter(features_relative[:,0],features_relative[:,1],s=0.1)

In [ ]:
plt.scatter(features_relative[:,0],features_relative[:,2],s=0.1)

In [ ]:
plt.scatter(features_relative[:,1],features_relative[:,2],s=0.1)

In [ ]:
plt.hist(np.sum(features[:,1:],axis=1).reshape(10000,1),bins=100);

In [ ]:
import plotly.express as px
fig = px.scatter_3d(x=features_relative[:,0],y=features_relative[:,1],z=features_relative[:,2],width=1000, height=1000,color=colors)

fig.update_layout(scene = dict(
                    xaxis_title='FITC',
                    yaxis_title='PE',
                    zaxis_title='APC'))

fig.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=0.3*2, y=0.2*2, z=0.8*2))

fig.update_traces(marker_size =2)
#fig.update_layout(scene_aspectmode='cube')

pio.renderers.default = "notebook"
fig.show()
fig.write_html(outpath+"\\gating.html")

In [ ]:
for i in df[df["Cell_Type"]==3]['Cell_ID']:
    plt.imshow(image_dict[i]['Ch6'],vmin=-50,vmax=50)
    plt.colorbar()
    plt.show()

In [ ]:
df